In [ ]:
import json
import requests

API_URL = "http://localhost:8000/check-plagiarism"
TEST_FILE = "test_cases.jsonl"

correct = 0
total = 0
failures = []

with open(TEST_FILE, "r") as f:
    for idx, line in enumerate(f, 1):
        entry = json.loads(line)
        payload = {
            "code": entry["code"],
            "top_k": entry.get("top_k", 10)
        }

        try:
            response = requests.post(API_URL, json=payload)
            result = response.json()

            predicted = result.get("plagiarism_detected", False)
            expected = entry["expected"]
            total += 1

            if predicted == expected:
                correct += 1
            else:
                failures.append({
                    "index": idx,
                    "expected": expected,
                    "predicted": predicted,
                    "code_snippet": entry["code"][:100] + "..."
                })

        except Exception as e:
            print(f"❌ Request error on entry {idx}: {e}")
            total += 1

# 📊 Output
accuracy = round((correct / total) * 100, 2)
print(f"\n✅ Accuracy: {correct}/{total} = {accuracy}%")

if failures:
    print(f"\n❌ {len(failures)} misclassifications:")
    for fail in failures:
        print(f"  Line {fail['index']}: Expected {fail['expected']}, Got {fail['predicted']}")
        print(f"    Snippet: {fail['code_snippet']}")